In [148]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from delta.tables import *

StatementMeta(, 4b9fd8f3-aa89-4389-b137-4323a7cd247b, 150, Finished, Available, Finished)

In [149]:
df = spark.read.table("innings_silver")

StatementMeta(, 4b9fd8f3-aa89-4389-b137-4323a7cd247b, 151, Finished, Available, Finished)

In [150]:
df_dimPlayer_silver = df.dropDuplicates(["batter"]).select(col('batter').alias('player_name'))

df_dimPlayer_silver = df_dimPlayer_silver.unionByName(df.dropDuplicates(["bowler"]).select(col('bowler').alias('player_name')))

df_dimPlayer_silver = df_dimPlayer_silver.unionByName(df.dropDuplicates(["non_striker"]).select(col('non_striker').alias('player_name')))

df_dimPlayer_silver = df_dimPlayer_silver.unionByName(df.dropDuplicates(["fielder1"]).select(col('fielder1').alias('player_name')))

df_dimPlayer_silver = df_dimPlayer_silver.unionByName(df.dropDuplicates(["fielder2"]).select(col('fielder2').alias('player_name')))

df_dimPlayer_silver = df_dimPlayer_silver.drop_duplicates().dropna()

StatementMeta(, 4b9fd8f3-aa89-4389-b137-4323a7cd247b, 152, Finished, Available, Finished)

In [151]:
df_dimPlayer_temp = spark.read.table("dimPlayer")

MAXPlayerId = df_dimPlayer_temp.select(coalesce(max(col("player_id")),lit(0)).alias("MAXPlayerId")).first()[0]

df_dimPlayer_gold = df_dimPlayer_silver.join(df_dimPlayer_temp,(df_dimPlayer_silver.player_name == df_dimPlayer_temp.player_name), "left_anti")

df_dimPlayer_gold = df_dimPlayer_gold.withColumn("player_id", monotonically_increasing_id() + MAXPlayerId+ 1)

StatementMeta(, 4b9fd8f3-aa89-4389-b137-4323a7cd247b, 153, Finished, Available, Finished)

In [152]:
deltaTable = DeltaTable.forPath(spark, 'Tables/dimplayer')
    
dfUpdates = df_dimPlayer_gold
    
deltaTable.alias('gold') \
  .merge(
    dfUpdates.alias('updates'),
    'gold.player_name = updates.player_name'
  ) \
   .whenMatchedUpdate(set =
    {
          
    }
  ) \
 .whenNotMatchedInsert(values =
    {
      "player_name": "updates.player_name",
      "player_id": "updates.player_id"
    }
  ) \
  .execute()

StatementMeta(, 4b9fd8f3-aa89-4389-b137-4323a7cd247b, 154, Finished, Available, Finished)

In [153]:
df_dimTeam_silver = df.dropDuplicates(["bat_team"]).select(col('bat_team').alias('team_name'))

df_dimTeam_silver = df_dimTeam_silver.unionByName(df.dropDuplicates(["bowl_team"]).select(col('bowl_team').alias('team_name')))

df_dimTeam_silver = df_dimTeam_silver.drop_duplicates().dropna()

StatementMeta(, 4b9fd8f3-aa89-4389-b137-4323a7cd247b, 155, Finished, Available, Finished)

In [154]:
df_dimTeam_temp = spark.read.table("dimTeam")

MAXTeamId = df_dimTeam_temp.select(coalesce(max(col("team_id")),lit(0)).alias("MAXTeamId")).first()[0]

df_dimTeam_gold = df_dimTeam_silver.join(df_dimTeam_temp,(df_dimTeam_silver.team_name == df_dimTeam_temp.name), "left_anti")

df_dimTeam_gold = df_dimTeam_gold.withColumn("team_id", monotonically_increasing_id() + MAXTeamId+ 1)

StatementMeta(, 4b9fd8f3-aa89-4389-b137-4323a7cd247b, 156, Finished, Available, Finished)

In [155]:
deltaTable = DeltaTable.forPath(spark, 'Tables/dimteam')
    
dfUpdates = df_dimTeam_gold
    
deltaTable.alias('gold') \
  .merge(
    dfUpdates.alias('updates'),
    'gold.name = updates.team_name'
  ) \
   .whenMatchedUpdate(set =
    {
          
    }
  ) \
 .whenNotMatchedInsert(values =
    {
      "name": "updates.team_name",
      "team_id": "updates.team_id"
    }
  ) \
  .execute()

StatementMeta(, 4b9fd8f3-aa89-4389-b137-4323a7cd247b, 157, Finished, Available, Finished)

In [156]:
df_dimPlayer_temp = spark.read.table("dimPlayer")
df_dimTeam_temp = spark.read.table("dimTeam")

df_dimMatch_silver = df.alias('df1') \
                       .join(df_dimPlayer_temp.alias('df2'), col('df1.player_of_match') == col('df2.player_name'), 'left') \
                       .join(df_dimTeam_temp.alias('df3'), col('df1.winner_team') == col('df3.name'), 'left') \
                       .join(df_dimTeam_temp.alias('df4'), col('df1.toss_winner_team') == col('df4.name'), 'left') \
                       .select(col('df1.file_name')
                           , col('df1.city')
                           , col('df1.date')
                           , col('df1.reserve_date')
                           , col('df1.season')
                           , col('df1.match_number')
                           , col('df1.stage')
                           , col('df3.team_id').alias('winner_team_id')
                           , col('df1.by_runs')
                           , col('df1.by_wickets')
                           , col('df2.player_id').alias('player_of_match_id')
                           , col('df1.toss_decision')
                           , col('df4.team_id').alias('toss_winner_team_id')
                           , col('df1.venue')) \
                       .dropDuplicates()

StatementMeta(, 4b9fd8f3-aa89-4389-b137-4323a7cd247b, 158, Finished, Available, Finished)

In [157]:
df_dimMatch_temp = spark.read.table("dimMatch")

MAXMatchId = df_dimMatch_temp.select(coalesce(max(col("match_id")),lit(0)).alias("MAXMatchId")).first()[0]

df_dimMatch_gold = df_dimMatch_silver.join(df_dimMatch_temp,(df_dimMatch_silver.file_name == df_dimMatch_temp.file_name), "left_anti")

df_dimMatch_gold = df_dimMatch_gold.withColumn("match_id", monotonically_increasing_id() + MAXMatchId+ 1)

StatementMeta(, 4b9fd8f3-aa89-4389-b137-4323a7cd247b, 159, Finished, Available, Finished)

In [158]:
deltaTable = DeltaTable.forPath(spark, "Tables/dimmatch")

dfUpdates = df_dimMatch_gold

# Perform the merge
deltaTable.alias("gold") \
  .merge(
    dfUpdates.alias("updates"),
    "gold.file_name = updates.file_name"
  ) \
  .whenMatchedUpdate(set={
    "match_id": "updates.match_id",
    "city": "updates.city",
    "date": "updates.date",
    "reserve_date": "updates.reserve_date",
    "season": "updates.season",
    "match_number": "updates.match_number",
    "stage": "updates.stage",
    "winner_team_id": "updates.winner_team_id",
    "by_runs": "updates.by_runs",
    "by_wickets": "updates.by_wickets",
    "player_of_match_id": "updates.player_of_match_id",
    "toss_decision": "updates.toss_decision",
    "toss_winner_team_id": "updates.toss_winner_team_id",
    "venue": "updates.venue"
  }) \
  .whenNotMatchedInsert(values={
    "match_id": "updates.match_id",
    "file_name": "updates.file_name",
    "city": "updates.city",
    "date": "updates.date",
    "reserve_date": "updates.reserve_date",
    "season": "updates.season",
    "match_number": "updates.match_number",
    "stage": "updates.stage",
    "winner_team_id": "updates.winner_team_id",
    "by_runs": "updates.by_runs",
    "by_wickets": "updates.by_wickets",
    "player_of_match_id": "updates.player_of_match_id",
    "toss_decision": "updates.toss_decision",
    "toss_winner_team_id": "updates.toss_winner_team_id",
    "venue": "updates.venue"
  }) \
  .execute()


StatementMeta(, 4b9fd8f3-aa89-4389-b137-4323a7cd247b, 160, Finished, Available, Finished)

In [159]:
df_dimDismissal_silver = df.dropDuplicates(["wicket"]).select(col('wicket').alias('type'))

df_dimDismissal_silver = df_dimDismissal_silver.drop_duplicates().dropna()

StatementMeta(, 4b9fd8f3-aa89-4389-b137-4323a7cd247b, 161, Finished, Available, Finished)

In [160]:
df_dimDismissal_temp = spark.read.table("dimDismissal")

MAXDismissalId = df_dimDismissal_temp.select(coalesce(max(col("dismissal_id")),lit(0)).alias("MAXDismissalId")).first()[0]

df_dimDismissal_gold = df_dimDismissal_silver.join(df_dimDismissal_temp,(df_dimDismissal_silver.type == df_dimDismissal_temp.type), "left_anti")

df_dimDismissal_gold = df_dimDismissal_gold.withColumn("dismissal_id", monotonically_increasing_id() + MAXDismissalId+ 1)

StatementMeta(, 4b9fd8f3-aa89-4389-b137-4323a7cd247b, 162, Finished, Available, Finished)

In [161]:
deltaTable = DeltaTable.forPath(spark, 'Tables/dimdismissal')
    
dfUpdates = df_dimDismissal_gold
    
deltaTable.alias('gold') \
  .merge(
    dfUpdates.alias('updates'),
    'gold.type = updates.type'
  ) \
   .whenMatchedUpdate(set =
    {
          
    }
  ) \
 .whenNotMatchedInsert(values =
    {
      "type": "updates.type",
      "dismissal_id": "updates.dismissal_id"
    }
  ) \
  .execute()

StatementMeta(, 4b9fd8f3-aa89-4389-b137-4323a7cd247b, 163, Finished, Available, Finished)

In [162]:
df_dimMatch_temp = spark.read.table("dimmatch")
df_dimDismissal_temp = spark.read.table("dimdismissal")

df_factInnings = df.alias('df1') \
                    .join(df_dimMatch_temp.alias("df2"), col('df2.file_name') == col('df1.file_name'), 'left') \
                    .join(df_dimTeam_temp.alias('df3'), col('df3.name') == col('df1.bat_team'), 'left') \
                    .join(df_dimTeam_temp.alias('df4'), col('df4.name') == col('df1.bowl_team'), 'left') \
                    .join(df_dimPlayer_temp.alias('df5'), col('df5.player_name') == col('df1.batter'), 'left') \
                    .join(df_dimPlayer_temp.alias('df6'), col('df5.player_name') == col('df1.bowler'), 'left') \
                    .join(df_dimPlayer_temp.alias('df7'), col('df6.player_name') == col('df1.non_striker'), 'left') \
                    .join(df_dimPlayer_temp.alias('df8'), col('df8.player_name') == col('df1.fielder1'), 'left') \
                    .join(df_dimPlayer_temp.alias('df9'), col('df9.player_name') == col('df1.fielder2'), 'left') \
                    .join(df_dimDismissal_temp.alias('df10'), col('df10.type') == col('df1.wicket'), 'left') \
                    .join(df_dimPlayer_temp.alias('df11'), col('df11.player_name') == col('df1.player_out'), 'left') \
                    .select(col('df2.match_id')
                        , col('df3.team_id').alias('bat_team_id')
                        , col('df4.team_id').alias('bowl_team_id')
                        , col('df1.over')  
                        , col('df1.delivery_id')
                        , col('df5.player_id').alias('batter_id')
                        , col('df6.player_id').alias('bowler_id')
                        , col('df7.player_id').alias('non_striker_id')
                        , col('bat_runs')
                        , col('extra_runs')
                        , col('total_runs')
                        , col('non_boundary')
                        , col('byes')
                        , col('legbyes')
                        , col('noballs')
                        , col('penalty')
                        , col('wides')
                        , col('df10.dismissal_id').alias('dismissal_id')
                        , col('df11.player_id').alias('player_out_id')
                        , col('df8.player_id').alias('fielder1_id')
                        , col('df9.player_id').alias('fielder2_id'))

StatementMeta(, 4b9fd8f3-aa89-4389-b137-4323a7cd247b, 164, Finished, Available, Finished)

In [163]:
from delta.tables import DeltaTable

deltaTable = DeltaTable.forPath(spark, "Tables/factinnings")

dfUpdates = df_factInnings

deltaTable.alias("fact") \
  .merge(
    dfUpdates.alias("updates"),
    """
    fact.match_id = updates.match_id AND
    fact.bat_team_id = updates.bat_team_id AND
    fact.bowl_team_id = updates.bowl_team_id AND
    fact.over = updates.over AND
    fact.delivery_id = updates.delivery_id
    """
  ) \
  .whenMatchedUpdate(set={
    ""
    "batter_id": "updates.batter_id",
    "bowler_id": "updates.bowler_id",
    "non_striker_id": "updates.non_striker_id",
    "bat_runs": "updates.bat_runs",
    "extra_runs": "updates.extra_runs",
    "total_runs": "updates.total_runs",
    "non_boundary": "updates.non_boundary",
    "byes": "updates.byes",
    "legbyes": "updates.legbyes",
    "noballs": "updates.noballs",
    "penalty": "updates.penalty",
    "wides": "updates.wides",
    "dismissal_id": "updates.dismissal_id",
    "player_out_id": "updates.player_out_id",
    "fielder1_id": "updates.fielder1_id",
    "fielder2_id": "updates.fielder2_id"
  }) \
  .whenNotMatchedInsert(values={
    "match_id": "updates.match_id",
    "bat_team_id": "updates.bat_team_id",
    "bowl_team_id": "updates.bowl_team_id",
    "over": "updates.over",
    "delivery_id": "updates.delivery_id",
    "batter_id": "updates.batter_id",
    "bowler_id": "updates.bowler_id",
    "non_striker_id": "updates.non_striker_id",
    "bat_runs": "updates.bat_runs",
    "extra_runs": "updates.extra_runs",
    "total_runs": "updates.total_runs",
    "non_boundary": "updates.non_boundary",
    "byes": "updates.byes",
    "legbyes": "updates.legbyes",
    "noballs": "updates.noballs",
    "penalty": "updates.penalty",
    "wides": "updates.wides",
    "dismissal_id": "updates.dismissal_id",
    "player_out_id": "updates.player_out_id",
    "fielder1_id": "updates.fielder1_id",
    "fielder2_id": "updates.fielder2_id"
  }) \
  .execute()


StatementMeta(, 4b9fd8f3-aa89-4389-b137-4323a7cd247b, 165, Finished, Available, Finished)